# Check of substitution conditions
This notebook applies conditions for examining when feasible substitution occurs. Here, only for the case of single production and with more industries than commodities ($c < i$) starting from a base SUT with also more industries than commodities.

Get the basis as described here https://discourse.julialang.org/t/how-to-obtain-a-basic-solution/1784/5 and here https://mathprogbasejl.readthedocs.io/en/latest/lpqcqp.html#getbasis. for the LCP, maybe work with PATH? https://github.com/chkwon/PATHSolver.jl

In [1]:
using LinearAlgebra
using JuMP
using GLPK
using XLSX
using DataFrames
#using Complementarity #not sure I need this one?
using PATHSolver
using Revise #not having to restart sessions

include("../src/SUT_structure2.jl") # Used for the SUT setup <sut = SUT.structure(...)>
include("../src/Constructs.jl") # Used to derive single-production systems from the SUT setup, e.g. as <itc = Constructs.ITC(sut)>
include("../src/Auxiliary.jl") # Includes some helper functions
include("../src/Model_data.jl") # Sets up the data structure for RCOT modelling based on SUT.structure or Constructs.construct
include("../src/RCOT_model.jl") # Builds and solves the RCOT models
include("../src/LP_models.jl"); # Builds and solves the LP models

In [68]:
import MathOptInterface as MOI

In [2]:
# Load the workbook containing SUT data, define the unit/type of the commodity data (monetary=default, physical, mixed), and optinally define factor availabilities.
wb = XLSX.readxlsx("../data/SUT_c-i+.xlsx")
t = "monetary"
ϕ = nothing;

In [3]:
# Load SUT data from workbook as dataframes into dictionary, where sheetnames serve as keys of the dictionary and where numerical data is declared as such (Float64).
dict = Dict()
for i in XLSX.sheetnames(wb)
    if i != "README"
        dict[i] = DataFrame(XLSX.gettable(wb[i]))
        dict[i][!,2:end] = convert.(Float64, dict[i][:,2:end])
    end
end

In [4]:
# Create a second dictionary that contains the SUT data as matrices without any column or row descriptors and where the keys are the same as of the original dictionary.
dict_m = Dict()
for i in keys(dict)
    dict_m[i] = Matrix(dict[i][:,2:end])
end

In [5]:
sut = SUT.structure(dict_m,t);

[ Info: A structure for supply-use elements was set up. Changing individual elements will not change others automatically.


In [ ]:
# itc = Constructs.ITC(sut)
# model_data = Model_data.IO(itc)
# model = io_rcot("primal", "rel", io_rcot_data);

Main.Constructs.construct([136.0; 96.0; 167.0;;], [51.0; 42.0; 31.0;;], [9.641472868217054 12.97168343454485 62.38684369723809; 31.21884735202492 15.000403984873474 7.780748663101605; 55.291530826631885 47.03340135357168 33.675067819796425], [0.07089318285453716 0.1351217024431755 0.37357391435471915; 0.22955034817665385 0.15625420817576535 0.04659130935988985; 0.4065553737252344 0.48993126409970494 0.20164711269339178], [1.59833317316061 0.7144586737028918 0.7896046410737875; 0.49661851260095824 1.4487520746555465 0.3169312490909092; 1.11870313292003 1.2528995188783316 1.8491729170383386], [11.41180057475428 4.148556760568855 11.939642664676864; 1.8050172265288542 2.3776251212691895 2.484024318868623; 2.3464303251306076 0.5423046132692892 5.259413209748251; 6.343125890506894 2.8536048675862875 12.231840670478245], [0.08391029834378147 0.043214132922592234 0.0714948662555501; 0.013272185489182752 0.024766928346554046 0.014874397118973792; 0.017253164155372116 0.005649006388221764 0.031

In [6]:
model_data = Model_data.SU(sut)

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


structure([0.0 10.0 … 38.0 22.0; 38.0 7.0 … 0.0 0.0; 49.0 59.0 … 17.0 11.0], [75.0 32.0 0.0; 34.0 50.0 18.0; … ; 0.0 12.0 47.0; 5.0 0.0 38.0], [10.0 1.6666666666666667 … 0.0 0.8333333333333334; 0.0 4.722222222222222 … 0.2777777777777778 1.3888888888888888; 1.1111111111111112 0.0 … 0.37037037037037035 0.7407407407407407; 2.380952380952381 3.333333333333333 … 0.9523809523809523 0.9523809523809523], [51.0; 42.0; 31.0;;], [1.2; 3.6; 2.7; 2.1;;], [107.0; 102.0; … ; 59.0; 43.0;;], [136.0; 96.0; 167.0;;], [27.499999999999996; 6.666666666666666; 8.148148148148147; 21.428571428571427;;], [27.499999999999996; 6.666666666666666; 8.148148148148147; 21.428571428571427;;], [0.0 0.09803921568627451 … 0.6440677966101694 0.5116279069767442; 0.35514018691588783 0.06862745098039216 … 0.0 0.0; 0.4579439252336448 0.5784313725490196 … 0.288135593220339 0.2558139534883721], [0.7009345794392523 0.3333333333333333 … 0.0 0.11627906976744186; 0.29906542056074764 0.49019607843137253 … 0.2033898305084746 0.0; 0.0 

In [7]:
# SU-RCOT - LPP
model = su_rcot("primal", "abs", model_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 123.99999999999999
value.(var_con) = [0.9999999999999993, 1.0, 1.0000000000000002, 0.9999999999999996, 0.9999999999999988]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [27.499999999999993, 6.666666666666666, 8.148148148148147, 21.428571428571427]


In [8]:
# SU-RCOT - LPD
model = su_rcot("dual", "abs", model_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(p) = [1.0000000000000007, 1.000000000000001, 1.0000000000000009]
value.(r) = [0.0, 3.984093603570996e-15, 0.0, 3.781093180473001e-15]
value.(profit_con) = [20.0, 26.0, 64.0, 4.0, 10.0]


In [9]:
# SU-RCOT - LPP
# Instantiate the model
primal = Model(GLPK.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));

ACT = size(model_data.V,1)

# Define the optimisation model
@variable(primal, z[i=1:ACT] >= 0)

@objective(primal, Min, sum(model_data.pii'*model_data.F*z))

primal[:con] = @constraints(primal, begin
    # Goods balance
    (model_data.V'-model_data.U)*z .>= model_data.e,
    (base_name = "goods")

    # Factor constraint,
    model_data.F*z .<= model_data.ϕ,
    (base_name = "factors")
end)

# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(z);

termination_status(primal) = MathOptInterface.OPTIMAL
primal_status(primal) = MathOptInterface.FEASIBLE_POINT
dual_status(primal) = MathOptInterface.FEASIBLE_POINT
objective_value(primal) = 123.99999999999999
value.(z) = [0.9999999999999993, 1.0, 1.0000000000000002, 0.9999999999999996, 0.9999999999999988]


In [133]:
# SU-RCOT - LPP
# Instantiate the model
primal = Model(GLPK.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));

# A-matrix
data = [model_data.V'-model_data.U; -model_data.F]
# c-vector
coeff = vec(model_data.pii'*model_data.F)
# b_vector
lower = Float64[]
append!(lower, model_data.e, -model_data.ϕ)

# Define the optimisation model
ACT = size(data,2)
@variable(primal, activity[i=1:ACT] >= 0)
@objective(primal, Min, sum(coeff' * activity))
@constraint(primal, data * activity .>= lower)

# Run the model and show results
optimize!(primal)
solution_summary(primal; verbose = true)

In [180]:
constr_v = all_constraints(primal; include_variable_in_set_constraints = true)
constr = all_constraints(primal; include_variable_in_set_constraints = false)

7-element Vector{ConstraintRef}:
 75 activity[1] + 24 activity[2] + 7 activity[3] - 38 activity[4] - 17 activity[5] >= 51
 -6 activity[1] + 43 activity[2] - 7 activity[3] + 12 activity[4] >= 42
 -49 activity[1] - 41 activity[2] + 64 activity[3] + 30 activity[4] + 27 activity[5] >= 31
 -10 activity[1] - 1.6666666666666667 activity[2] - 15 activity[3] - 0.8333333333333334 activity[5] >= -27.499999999999996
 -4.722222222222222 activity[2] - 0.2777777777777778 activity[3] - 0.2777777777777778 activity[4] - 1.3888888888888888 activity[5] >= -6.666666666666666
 -1.1111111111111112 activity[1] - 5.925925925925926 activity[3] - 0.37037037037037035 activity[4] - 0.7407407407407407 activity[5] >= -8.148148148148147
 -2.380952380952381 activity[1] - 3.333333333333333 activity[2] - 13.809523809523808 activity[3] - 0.9523809523809523 activity[4] - 0.9523809523809523 activity[5] >= -21.428571428571427

In [149]:
A_init = [data -I]

7×12 Matrix{Float64}:
  75.0       24.0        7.0       …   0.0   0.0   0.0   0.0   0.0   0.0
  -6.0       43.0       -7.0          -1.0   0.0   0.0   0.0   0.0   0.0
 -49.0      -41.0       64.0           0.0  -1.0   0.0   0.0   0.0   0.0
 -10.0       -1.66667  -15.0           0.0   0.0  -1.0   0.0   0.0   0.0
  -0.0       -4.72222   -0.277778      0.0   0.0   0.0  -1.0   0.0   0.0
  -1.11111   -0.0       -5.92593   …   0.0   0.0   0.0   0.0  -1.0   0.0
  -2.38095   -3.33333  -13.8095        0.0   0.0   0.0   0.0   0.0  -1.0

In [200]:
basic_cols = Bool[]

for act in 1:ACT
    if MOI.get(primal, MOI.VariableBasisStatus(), activity[act]) == MOI.BASIC
        val = 1
    else
        val = 0
    end
    append!(basic_cols,val)
end

for con in 1:length(constr)
    if MOI.get(primal, MOI.ConstraintBasisStatus(), constr[con]) == MOI.BASIC
        val = 1
    else
        val = 0
    end
    append!(basic_cols,val)
end

In [203]:
Basis = A_init[:, basic_cols]

7×7 Matrix{Float64}:
  75.0       24.0        7.0       -38.0       -17.0        0.0   0.0
  -6.0       43.0       -7.0        12.0         0.0        0.0   0.0
 -49.0      -41.0       64.0        30.0        27.0        0.0   0.0
 -10.0       -1.66667  -15.0        -0.0        -0.833333  -1.0   0.0
  -0.0       -4.72222   -0.277778   -0.277778   -1.38889    0.0   0.0
  -1.11111   -0.0       -5.92593    -0.37037    -0.740741   0.0  -1.0
  -2.38095   -3.33333  -13.8095     -0.952381   -0.952381   0.0   0.0

In [ ]:
# Condition 1 - LPP: LP
# Instantiate the model
primal = Model(GLPK.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));

# A-matrix
B_omega = Basis
# c-vector
unit = zeros()
unit[j] = 1
# b_vector
k = -100
m = 10
w_bar = m .* lower

# Define the optimisation model
ACT = size(Basis,2)
BAS = size(Basis,1)
@variable(primal, B_activity[i=1:ACT] >= 0)
@variable(primal, w[i=1:ACT] >= 0)

@objective(primal, Min, sum(unit' * B_activity))
@constraint(primal, B_omega * B_activity .== w)
@constraint(primal, B_activity .>= k * ones(BAS))
@constraint(primal, w .>= 0)
@constraint(primal, w .<= w_bar)

# Run the model and show results
optimize!(primal)
solution_summary(primal; verbose = true)

In [ ]:
# Condition 1 - LPP: LD
# Instantiate the model
dual = Model(GLPK.Optimizer)
(@isdefined dual) && (dual = nothing; dual = Model(GLPK.Optimizer));

# Define the optimisation model
ACT = size(Basis,2)
BAS = size(Basis,1)
@variable(dual, lambda[i=1:BAS] >= 0)
@variable(dual, rho[i=1:ACT] >= 0)
@variable(dual, beta[i=1:BAS])

@objective(dual, Max, sum(rho' * k * unit - lambda' * w_bar))
@constraint(dual, B_omega' * beta - rho .== -unit)
@constraint(dual, lambda - beta .>= 0)

# Run the model and show results
optimize!(dual)
solution_summary(dual; verbose = true)

In [ ]:
# Condition 1 - LCP
# Instantiate the model
compl = Model(PATHSolver.Optimizer)
(@isdefined compl) && (compl = nothing; compl = Model(GLPK.Optimizer));

# A-matrix
B_omega = Basis
# c-vector
unit = zeros()
unit[j] = 1
# b_vector
k = -100
m = 10
w_bar = m .* lower

# Define the optimisation model
ACT = size(Basis,2)
BAS = size(Basis,1)
FAC = 
@variable(primal, B_activity[i=1:ACT] >= 0)
@variable(primal, w[i=1:ACT] >= 0)
@variable(dual, lambda[i=1:BAS] >= 0)
@variable(dual, rho[i=1:ACT] >= 0)
@variable(dual, beta[i=1:BAS])

@constraint(compl, c1, beta ⟂ vec(B_omega * B_activity - w))
@constraint(compl, c2, B_activity ⟂ vec(B_omega' * beta - rho - unit))
@constraint(compl, c3, rho ⟂ vec(B_activity - k * ones(ACT)))
@constraint(compl, c4, lambda ⟂ vec(w_bar - w))
@constraint(compl, c5, w ⟂ vec(lambda - beta))

# Run the model and show results
optimize!(compl)
solution_summary(compl; verbose = true)

Use ParametricOptInterface when dealing with uni vector and changes of it? https://jump.dev/ParametricOptInterface.jl

In [206]:
value.(activity)

5-element Vector{Float64}:
 0.9999999999999993
 1.0
 1.0000000000000002
 0.9999999999999996
 0.9999999999999988

In [214]:
rank(data)

5

In [216]:
norm(Basis,2)

143.07057359031882

In [217]:
det(Basis)

-457429.6296296293

In [219]:
nullspace(Basis)

7×0 Matrix{Float64}

In [212]:
inv(Basis)*lower

7-element Vector{Float64}:
  0.9999999999999964
  0.9999999999999996
  1.0000000000000004
  1.0
  0.9999999999999963
 -7.224125172668125e-16
 -3.124597976476786e-16

In [183]:
length(constr)

enumerate(ConstraintRef[75 activity[1] + 24 activity[2] + 7 activity[3] - 38 activity[4] - 17 activity[5] >= 51, -6 activity[1] + 43 activity[2] - 7 activity[3] + 12 activity[4] >= 42, -49 activity[1] - 41 activity[2] + 64 activity[3] + 30 activity[4] + 27 activity[5] >= 31, -10 activity[1] - 1.6666666666666667 activity[2] - 15 activity[3] - 0.8333333333333334 activity[5] >= -27.499999999999996, -4.722222222222222 activity[2] - 0.2777777777777778 activity[3] - 0.2777777777777778 activity[4] - 1.3888888888888888 activity[5] >= -6.666666666666666, -1.1111111111111112 activity[1] - 5.925925925925926 activity[3] - 0.37037037037037035 activity[4] - 0.7407407407407407 activity[5] >= -8.148148148148147, -2.380952380952381 activity[1] - 3.333333333333333 activity[2] - 13.809523809523808 activity[3] - 0.9523809523809523 activity[4] - 0.9523809523809523 activity[5] >= -21.428571428571427])

In [179]:
basic_cols

5-element Vector{Int64}:
 1
 1
 1
 1
 1

In [164]:
MOI.get(primal, MOI.VariableBasisStatus(), activity[5])

BASIC::BasisStatusCode = 0

In [155]:
MOI.get(primal, MOI.ConstraintBasisStatus(), constr[7])

NONBASIC::BasisStatusCode = 1

In [156]:
std_form = JuMP._standard_form_matrix(primal)
basis = JuMP._standard_form_basis(primal, std_form)

(variables = MathOptInterface.BasisStatusCode[MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC], bounds = MathOptInterface.BasisStatusCode[MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC], constraints = MathOptInterface.BasisStatusCode[MathOptInterface.NONBASIC_AT_LOWER, MathOptInterface.NONBASIC_AT_LOWER, MathOptInterface.NONBASIC_AT_LOWER, MathOptInterface.BASIC, MathOptInterface.NONBASIC_AT_LOWER, MathOptInterface.BASIC, MathOptInterface.NONBASIC_AT_LOWER], basic_cols = Bool[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0])

In [141]:
show(stdout, "text/plain", std_form.A)

7×12 SparseArrays.SparseMatrixCSC{Float64, Int64} with 38 stored entries:
  75.0       24.0        7.0       -38.0       -17.0       -1.0    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  -6.0       43.0       -7.0        12.0          ⋅          ⋅   -1.0    ⋅     ⋅     ⋅     ⋅     ⋅ 
 -49.0      -41.0       64.0        30.0        27.0         ⋅     ⋅   -1.0    ⋅     ⋅     ⋅     ⋅ 
 -10.0       -1.66667  -15.0          ⋅         -0.833333    ⋅     ⋅     ⋅   -1.0    ⋅     ⋅     ⋅ 
    ⋅        -4.72222   -0.277778   -0.277778   -1.38889     ⋅     ⋅     ⋅     ⋅   -1.0    ⋅     ⋅ 
  -1.11111     ⋅        -5.92593    -0.37037    -0.740741    ⋅     ⋅     ⋅     ⋅     ⋅   -1.0    ⋅ 
  -2.38095   -3.33333  -13.8095     -0.952381   -0.952381    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   -1.0

In [157]:
std_form.lower

12-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
  51.0
  42.0
  31.0
 -27.499999999999996
  -6.666666666666666
  -8.148148148148147
 -21.428571428571427

In [158]:
basis.basic_cols

12-element BitVector:
 1
 1
 1
 1
 1
 0
 0
 0
 1
 0
 1
 0

In [144]:
basis.constraints

7-element Vector{MathOptInterface.BasisStatusCode}:
 NONBASIC_AT_LOWER::BasisStatusCode = 2
 NONBASIC_AT_LOWER::BasisStatusCode = 2
 NONBASIC_AT_LOWER::BasisStatusCode = 2
 BASIC::BasisStatusCode = 0
 NONBASIC_AT_LOWER::BasisStatusCode = 2
 BASIC::BasisStatusCode = 0
 NONBASIC_AT_LOWER::BasisStatusCode = 2

In [145]:
basis.variables

5-element Vector{MathOptInterface.BasisStatusCode}:
 BASIC::BasisStatusCode = 0
 BASIC::BasisStatusCode = 0
 BASIC::BasisStatusCode = 0
 BASIC::BasisStatusCode = 0
 BASIC::BasisStatusCode = 0

In [46]:
# SU-RCOT - LCP
compl = Model(PATHSolver.Optimizer)

ACT = size(model_data.V,1)
COM = size(model_data.V,2)
FAC = size(model_data.F,1)

# Define the optimisation model
@variable(compl, z_star[1:ACT] >= 0)
@variable(compl, p_star[1:COM] >= 0)
@variable(compl, r_star[1:FAC] >= 0)

@constraint(compl, c1, vec((model_data.pii + r_star)'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(compl, c2, vec((model_data.V' - model_data.U)*(z_star) - model_data.e) ⟂ p_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ r_star)

# Solve
optimize!(compl)
solution_summary(compl; verbose = true)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.4596e+02             0.0e+00 (f[    6])
    1     9     3     3 1.4538e+02  1.2e-02    1.5e+00 (f[    6])
    2    19     0     3 1.4528e+02  1.9e-03    1.3e+00 (f[    6])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    20     3 1.4528e+02           I 1.2e+00 1.0e+02 (f[    6])
    1     8    21     4 1.3706e+00  1.0e+00 SM 4.7e-01 6.2e-01 (f[    3])
    2     2    22     5 6.6832e-02  1.0e+00 SO 1.4e-01 4.7e-02 (f[    4])
    3     1    23     6 1.9357e-04  1.0e+00 SO 6.7e-03 1.3e-04 (f[    9])
    4     1    24     7 1.7722e-09  1.0e+00 SO 1.9e-05 8.8e-10 (f[    4])

Major Iterations. . . . 4
Minor Iterations. . . . 12
Restarts. . . . . . . . 0
Crash Iterations. . . . 2
Gradient 

In [47]:
solution_summary(compl; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 1.07105e+00
    p_star[2] : 1.07501e+00
    p_star[3] : 1.06882e+00
    r_star[1] : 1.03179e-01
    r_star[2] : 2.69490e-01
    r_star[3] : 0.00000e+00
    r_star[4] : 1.99423e-01
    z_star[1] : 1.00000e+00
    z_star[2] : 1.00000e+00
    z_star[3] : 1.00000e+00
    z_star[4] : 1.00000e+00
    z_star[5] : 1.00000e+00

* Work counters
  Solve time (sec)   : 0.00000e+00


In [25]:
MOI.get(primal, MOI.VariableBasisStatus())

LoadError: MethodError: no method matching get(::Model, ::MathOptInterface.VariableBasisStatus)

[0mClosest candidates are:
[0m  get(::GenericModel, ::MathOptInterface.AbstractVariableAttribute, [91m::GenericVariableRef[39m)
[0m[90m   @[39m [36mJuMP[39m [90mC:\Users\maximiko\.julia\packages\JuMP\jZvaU\src\[39m[90m[4moptimizer_interface.jl:661[24m[39m
[0m  get([91m::MathOptInterface.Bridges.AbstractBridgeOptimizer[39m, ::MathOptInterface.AbstractVariableAttribute, [91m::Vector{MathOptInterface.VariableIndex}[39m)
[0m[90m   @[39m [35mMathOptInterface[39m [90mC:\Users\maximiko\.julia\packages\MathOptInterface\864xP\src\Bridges\[39m[90m[4mbridge_optimizer.jl:1200[24m[39m
[0m  get([91m::MathOptInterface.Bridges.AbstractBridgeOptimizer[39m, ::MathOptInterface.AbstractVariableAttribute, [91m::MathOptInterface.VariableIndex[39m)
[0m[90m   @[39m [35mMathOptInterface[39m [90mC:\Users\maximiko\.julia\packages\MathOptInterface\864xP\src\Bridges\[39m[90m

In [ ]:
MOI.get(primal, MOI.ConstraintsBasisStatus())

LoadError: MethodError: no method matching get(::Model, ::MathOptInterface.VariableBasisStatus)

[0mClosest candidates are:
[0m  get(::GenericModel, ::MathOptInterface.AbstractVariableAttribute, [91m::GenericVariableRef[39m)
[0m[90m   @[39m [36mJuMP[39m [90mC:\Users\maximiko\.julia\packages\JuMP\jZvaU\src\[39m[90m[4moptimizer_interface.jl:661[24m[39m
[0m  get([91m::MathOptInterface.Bridges.AbstractBridgeOptimizer[39m, ::MathOptInterface.AbstractVariableAttribute, [91m::Vector{MathOptInterface.VariableIndex}[39m)
[0m[90m   @[39m [35mMathOptInterface[39m [90mC:\Users\maximiko\.julia\packages\MathOptInterface\864xP\src\Bridges\[39m[90m[4mbridge_optimizer.jl:1200[24m[39m
[0m  get([91m::MathOptInterface.Bridges.AbstractBridgeOptimizer[39m, ::MathOptInterface.AbstractVariableAttribute, [91m::MathOptInterface.VariableIndex[39m)
[0m[90m   @[39m [35mMathOptInterface[39m [90mC:\Users\maximiko\.julia\packages\MathOptInterface\864xP\src\Bridges\[39m[90m

In [35]:
MOI.get(GLPK.Optimizer(), MOI.ResultCount())

0

In [36]:
backend(model)

MOIU.CachingOptimizer{MOIB.LazyBridgeOptimizer{GLPK.Optimizer}, MOIU.UniversalFallback{MOIU.Model{Float64}}}
in state ATTACHED_OPTIMIZER
in mode AUTOMATIC
with model cache MOIU.UniversalFallback{MOIU.Model{Float64}}
  fallback for MOIU.Model{Float64}
with optimizer MOIB.LazyBridgeOptimizer{GLPK.Optimizer}
  with 0 variable bridges
  with 0 constraint bridges
  with 0 objective bridges
  with inner model A GLPK model

In [37]:
MOI.ResultStatusCode

Enum MathOptInterface.ResultStatusCode:
NO_SOLUTION = 0
FEASIBLE_POINT = 1
NEARLY_FEASIBLE_POINT = 2
INFEASIBLE_POINT = 3
INFEASIBILITY_CERTIFICATE = 4
NEARLY_INFEASIBILITY_CERTIFICATE = 5
REDUCTION_CERTIFICATE = 6
NEARLY_REDUCTION_CERTIFICATE = 7
UNKNOWN_RESULT_STATUS = 8
OTHER_RESULT_STATUS = 9

In [41]:
MOI.VariableBasisStatus

MathOptInterface.VariableBasisStatus

In [45]:
MOI.get(primal, MOI.PrimalStatus())

FEASIBLE_POINT::ResultStatusCode = 1

In [48]:
MOI.get(primal, MOI.TerminationStatus())

OPTIMAL::TerminationStatusCode = 1

In [49]:
MOI.get(primal, MOI.DualStatus())

FEASIBLE_POINT::ResultStatusCode = 1

In [51]:
MOI.get(primal, MOI.SolveTimeSec())

0.0

In [55]:
model_data.ϕ'*model_data.F

1×5 Matrix{Float64}:
 335.074  148.743  758.556  25.2778  58.6198

In [53]:
MOI.get(primal, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}())

0.0 + 20.0 MOI.VariableIndex(1) + 26.0 MOI.VariableIndex(2) + 64.0 MOI.VariableIndex(3) + 10.0 MOI.VariableIndex(5) + 4.0 MOI.VariableIndex(4)

In [81]:
MOI.get(primal, MOI.ConstraintBasisStatus(), primal[:con][2][2])

NONBASIC::BasisStatusCode = 1

In [69]:
MOI.get(primal, MOI.VariableBasisStatus(), z[5])

BASIC::BasisStatusCode = 0

In [71]:
MOI.get(model, MOI.ConstraintBasisStatus(), LowerBoundRef(x))

LoadError: UndefVarError: `x` not defined

In [94]:
MOI.get(primal, MOI.ConstraintBasisStatus(), constr[12])

NONBASIC::BasisStatusCode = 1

In [80]:
constr = all_constraints(primal; include_variable_in_set_constraints = true)

12-element Vector{ConstraintRef}:
 factors : 10 z[1] + 1.6666666666666667 z[2] + 15 z[3] + 0.8333333333333334 z[5] <= 27.499999999999996
 factors : 4.722222222222222 z[2] + 0.2777777777777778 z[3] + 0.2777777777777778 z[4] + 1.3888888888888888 z[5] <= 6.666666666666666
 factors : 1.1111111111111112 z[1] + 5.925925925925926 z[3] + 0.37037037037037035 z[4] + 0.7407407407407407 z[5] <= 8.148148148148147
 factors : 2.380952380952381 z[1] + 3.333333333333333 z[2] + 13.809523809523808 z[3] + 0.9523809523809523 z[4] + 0.9523809523809523 z[5] <= 21.428571428571427
 z[1] >= 0
 z[2] >= 0
 z[3] >= 0
 z[4] >= 0
 z[5] >= 0
 goods : 75 z[1] + 24 z[2] + 7 z[3] - 38 z[4] - 17 z[5] >= 51
 goods : -6 z[1] + 43 z[2] - 7 z[3] + 12 z[4] >= 42
 goods : -49 z[1] - 41 z[2] + 64 z[3] + 30 z[4] + 27 z[5] >= 31

In [97]:
MOI.get(primal, MOI.NumberOfVariables())

5

In [58]:
MOI.ConstraintBasisStatus

MathOptInterface.ConstraintBasisStatus

In [61]:
foo = all_constraints(primal, VariableRef, MOI.Integer)

ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.VariableIndex, MathOptInterface.Integer}, ScalarShape}[]

In [62]:
foo

ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.VariableIndex, MathOptInterface.Integer}, ScalarShape}[]

In [65]:
num_constraints(primal, AffExpr, MOI.LessThan{Float64})

4

In [66]:
num_constraints(primal, AffExpr, MOI.GreaterThan{Float64})

3

In [67]:
num_constraints(primal, VariableRef, MOI.GreaterThan{Float64})

5

In [68]:
num_constraints(primal, VariableRef, MOI.LessThan{Float64})

0

In [69]:
print(primal)

In [70]:
MOI.get(primal, MOI.ObjectiveSense())

MIN_SENSE::OptimizationSense = 0

In [17]:
using HiGHS

In [37]:
# SU-RCOT - LPP
# Instantiate the model
#solver = optimizer_with_attributes(HiGHS.Optimizer, "solver" => "simplex")
#primal = Model(solver)
#primal = direct_model(solver)
solver = optimizer_with_attributes(HiGHS.Optimizer)
primal = direct_model(solver)
set_attribute(primal, "solver", "simplex")

#(@isdefined primal) && (primal = nothing; primal = Model(solver));

ACT = size(model_data.V,1)

# Define the optimisation model
@variable(primal, z[i=1:ACT] >= 0)

@objective(primal, Min, sum(model_data.pii'*model_data.F*z))

primal[:con] = @constraints(primal, begin
    # Goods balance
    (model_data.V'-model_data.U)*z .>= model_data.e,
    (base_name = "goods")

    # Factor constraint,
    model_data.F*z .<= model_data.ϕ,
    (base_name = "factors")
end)

# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(z);

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
7 rows, 5 cols, 31 nonzeros
7 rows, 5 cols, 31 nonzeros
Presolve : Reductions: rows 7(-0); columns 5(-0); elements 31(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(37.625) 0s
          6     1.2400000000e+02 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 6
Objective value     :  1.2400000000e+02
HiGHS run time      :          0.00
termination_status(primal) = MathOptInterface.OPTIMAL
primal_status(primal) = MathOptInterface.FEASIBLE_POINT
dual_status(primal) = MathOptInterface.FEASIBLE_POINT
objective_value(primal) = 123.99999999999999
value.(z) = [0.9999999999999974, 1.0000000000000007, 1.0000000000000009, 0.9999999999999978, 0.9999999999999978]


In [31]:
solution_summary(primal; result = 1)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.24000e+02
  Objective bound    : 0.00000e+00
  Relative gap       : Inf
  Dual objective value : 1.24000e+02

* Work counters
  Solve time (sec)   : 0.00000e+00
  Simplex iterations : 6
  Barrier iterations : 0
  Node count         : -1


In [71]:
HiGHS.Highs_version()

Ptr{Int8} @0x000000007101a022

In [20]:
highs = HiGHS.Highs_create()

Ptr{Nothing} @0x00000185979d6010

In [38]:
HiGHS._ConstraintInfo()

LoadError: MethodError: no method matching HiGHS._ConstraintInfo()

[0mClosest candidates are:
[0m  HiGHS._ConstraintInfo([91m::Union{MathOptInterface.EqualTo{Float64}, MathOptInterface.GreaterThan{Float64}, MathOptInterface.Interval{Float64}, MathOptInterface.LessThan{Float64}}[39m)
[0m[90m   @[39m [36mHiGHS[39m [90mC:\Users\maximiko\.julia\packages\HiGHS\zTMN2\src\[39m[90m[4mMOI_wrapper.jl:154[24m[39m
[0m  HiGHS._ConstraintInfo([91m::String[39m, [91m::Int32[39m, [91m::HiGHS._RowType[39m, [91m::Float64[39m, [91m::Float64[39m)
[0m[90m   @[39m [36mHiGHS[39m [90mC:\Users\maximiko\.julia\packages\HiGHS\zTMN2\src\[39m[90m[4mMOI_wrapper.jl:145[24m[39m
[0m  HiGHS._ConstraintInfo([91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m)
[0m[90m   @[39m [36mHiGHS[39m [90mC:\Users\maximiko\.julia\packages\HiGHS\zTMN2\src\[39m[90m[4mMOI_wrapper.jl:145[24m[39m


In [39]:
HiGHS.Highs_getBasis(primal)

LoadError: MethodError: no method matching Highs_getBasis(::Model)

[0mClosest candidates are:
[0m  Highs_getBasis(::Any, [91m::Any[39m, [91m::Any[39m)
[0m[90m   @[39m [36mHiGHS[39m [90mC:\Users\maximiko\.julia\packages\HiGHS\zTMN2\src\gen\[39m[90m[4mlibhighs.jl:831[24m[39m


In [10]:
primal

A JuMP Model
Minimization problem with:
Variables: 5
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: con, z

In [45]:
HiGHS.Highs_getObjectiveSense(highs, kHighsObjSenseMinimize)

LoadError: MethodError: no method matching unsafe_convert(::Type{Ptr{Int32}}, ::Int32)

[0mClosest candidates are:
[0m  unsafe_convert(::Type{Ptr{T}}, [91m::SharedArrays.SharedArray{T}[39m) where T
[0m[90m   @[39m [32mSharedArrays[39m [90mC:\Users\maximiko\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\SharedArrays\src\[39m[90m[4mSharedArrays.jl:361[24m[39m
[0m  unsafe_convert(::Type{Ptr{T}}, [91m::SharedArrays.SharedArray[39m) where T
[0m[90m   @[39m [32mSharedArrays[39m [90mC:\Users\maximiko\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\SharedArrays\src\[39m[90m[4mSharedArrays.jl:362[24m[39m
[0m  unsafe_convert(::Type{Ptr{T}}, [91m::SparseArrays.ReadOnly[39m) where T
[0m[90m   @[39m [33mSparseArrays[39m [90mC:\Users\maximiko\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\SparseArrays\src\[39m[90m[4mreadonly.jl:23[24m[39m
[0m  ...


In [49]:
HiGHS.Highs_versionMajor()

1

In [18]:
rm("model.lp", recursive=true)

In [19]:
write_to_file(primal, "model.lp")

In [21]:
HiGHS.Highs_readModel(highs, "model.lp")

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms


0

In [22]:
HiGHS.Highs_getModelStatus(highs)

0

In [57]:
senseP = Ref{Cint}(0)

Base.RefValue{Int32}(0)

In [58]:
HiGHS.Highs_getObjectiveSense(highs, senseP)

0

In [62]:
senseP[]# == kHighsObjSenseMinimize

1

In [12]:
src = primal

A JuMP Model
Minimization problem with:
Variables: 5
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: con, z

In [13]:
dest = MOI.FileFormats.Model(format = MOI.FileFormats.FORMAT_LP)

A .LP-file model

In [14]:
MOI.copy_to(dest, src);

MathOptInterface.Utilities.IndexMap with 17 entries:
  MOI.VariableIndex(1)      => MOI.VariableIndex(1)
  MOI.VariableIndex(2)      => MOI.VariableIndex(2)
  MOI.VariableIndex(3)      => MOI.VariableIndex(3)
  MOI.VariableIndex(4)      => MOI.VariableIndex(4)
  MOI.VariableIndex(5)      => MOI.VariableIndex(5)
  ConstraintIndex{ScalarAf… => ConstraintIndex{ScalarAffineFunction{Float64}, L…
  ConstraintIndex{ScalarAf… => ConstraintIndex{ScalarAffineFunction{Float64}, L…
  ConstraintIndex{ScalarAf… => ConstraintIndex{ScalarAffineFunction{Float64}, L…
  ConstraintIndex{ScalarAf… => ConstraintIndex{ScalarAffineFunction{Float64}, L…
  ConstraintIndex{Variable… => ConstraintIndex{VariableIndex, GreaterThan{Float…
  ConstraintIndex{Variable… => ConstraintIndex{VariableIndex, GreaterThan{Float…
  ConstraintIndex{Variable… => ConstraintIndex{VariableIndex, GreaterThan{Float…
  ConstraintIndex{Variable… => ConstraintIndex{VariableIndex, GreaterThan{Float…
  ConstraintIndex{Variable… => Constrai

In [41]:
io = IOBuffer();

In [16]:
#write(io, dest)

In [29]:
write(io, src)

In [17]:
seekstart(io);

In [18]:
highs = HiGHS.Highs_create();

Ptr{Nothing} @0x00000214b2fe8060

In [42]:
typeof(highs)

Ptr{Nothing}

In [39]:
read!(io, highs)

Ptr{Nothing} @0x00000214b2fe8060

In [31]:
HiGHS.Highs_getModelStatus(highs)

6

In [30]:
HiGHS.Highs_readModel(highs, io)

LoadError: MethodError: no method matching unsafe_convert(::Type{Ptr{Int8}}, ::IOBuffer)

[0mClosest candidates are:
[0m  unsafe_convert(::Union{Cstring, Type{Ptr{Int8}}, Type{Ptr{UInt8}}}, [91m::LaTeXStrings.LaTeXString[39m)
[0m[90m   @[39m [36mLaTeXStrings[39m [90mC:\Users\maximiko\.julia\packages\LaTeXStrings\pJ7vn\src\[39m[90m[4mLaTeXStrings.jl:129[24m[39m
[0m  unsafe_convert(::Type{Ptr{T}}, [91m::SharedArrays.SharedArray{T}[39m) where T
[0m[90m   @[39m [32mSharedArrays[39m [90mC:\Users\maximiko\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\SharedArrays\src\[39m[90m[4mSharedArrays.jl:361[24m[39m
[0m  unsafe_convert(::Type{Ptr{T}}, [91m::SharedArrays.SharedArray[39m) where T
[0m[90m   @[39m [32mSharedArrays[39m [90mC:\Users\maximiko\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\SharedArrays\src\[39m[90m[4mSharedArrays.jl:362[24m[39m
[0m  ...


In [23]:
HiGHS.Highs_run(highs)

Presolving model
7 rows, 5 cols, 31 nonzeros
7 rows, 5 cols, 31 nonzeros
Presolve : Reductions: rows 7(-0); columns 5(-0); elements 31(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(37.625) 0s
          6     1.2400000000e+02 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 6
Objective value     :  1.2400000000e+02
HiGHS run time      :          0.01


0

In [68]:
col_value = zeros(Cdouble, 10);

In [69]:
HiGHS.Highs_getSolution(highs, col_value, C_NULL, C_NULL, C_NULL)

0

In [70]:
col_value

10-element Vector{Float64}:
 0.9999999999999967
 1.0000000000000007
 1.000000000000001
 0.9999999999999974
 0.9999999999999962
 0.0
 0.0
 0.0
 0.0
 0.0

In [45]:
HiGHS.HiGHS.Highs_getNumRow(highs)

7

In [73]:
HiGHS.HiGHS.Highs_getNumCol(highs)

5

In [74]:
HiGHS.HiGHS.Highs_getNumNz(highs)

31

In [75]:
HiGHS.HiGHS.Highs_getNumOptions(highs)

65

In [76]:
HiGHS.Highs_getObjectiveValue(highs)

123.99999999999999

In [24]:
nrows = HiGHS.HiGHS.Highs_getNumRow(highs)

7

In [62]:
basic_variables = zeros(nrows)

7-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [25]:
basic_variables = fill(NaN, nrows)

7-element Vector{Float64}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [63]:
#citArr = map(1:nrows) do i
    x1 = nothing
    y1 = nothing
    (x1, y1)
  end

7-element Vector{Tuple{Nothing, Nothing}}:
 (nothing, nothing)
 (nothing, nothing)
 (nothing, nothing)
 (nothing, nothing)
 (nothing, nothing)
 (nothing, nothing)
 (nothing, nothing)

In [23]:
#arr = Array{Union{Nothing, Any}}(nothing, nrows)

7-element Vector{Any}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [56]:
highs

Ptr{Nothing} @0x00000214b2fe8060

I noticed that getBasicVariables returns an array, presumably filled with tuples. The docs say that an array is returned with {1,-1} as an entry for example..

In [24]:
HiGHS.Highs_getBasicVariables(highs, arr)#basic_variables)

0

In [25]:
arr

: 

In [26]:
HiGHS.Highs_getBasicVariables(highs, basic_variables)

0

In [27]:
basic_variables

7-element Vector{Float64}:
   1.06099789543e-313
   8.4879831624e-314
   5.0e-324
 NaN
 NaN
 NaN
 NaN

I should probably pass "basic_variables" as an array?!

In [54]:
println(dict_m["V"])

[75.0 32.0 0.0; 34.0 50.0 18.0; 22.0 2.0 64.0; 0.0 12.0 47.0; 5.0 0.0 38.0]


In [61]:
show(stdout, "text/plain", std_form.A)

7×12 SparseArrays.SparseMatrixCSC{Float64, Int64} with 38 stored entries:
  10.0        1.66667  15.0          ⋅          0.833333  -1.0    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
    ⋅         4.72222   0.277778    0.277778    1.38889     ⋅   -1.0    ⋅     ⋅     ⋅     ⋅     ⋅ 
   1.11111     ⋅        5.92593     0.37037     0.740741    ⋅     ⋅   -1.0    ⋅     ⋅     ⋅     ⋅ 
   2.38095    3.33333  13.8095      0.952381    0.952381    ⋅     ⋅     ⋅   -1.0    ⋅     ⋅     ⋅ 
  75.0       24.0       7.0       -38.0       -17.0         ⋅     ⋅     ⋅     ⋅   -1.0    ⋅     ⋅ 
  -6.0       43.0      -7.0        12.0          ⋅          ⋅     ⋅     ⋅     ⋅     ⋅   -1.0    ⋅ 
 -49.0      -41.0      64.0        30.0        27.0         ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   -1.0

In [101]:
std_form.lower

12-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
 -Inf
 -Inf
 -Inf
 -Inf
  51.0
  42.0
  31.0

In [38]:
std_form = JuMP._standard_form_matrix(primal)
basis = JuMP._standard_form_basis(primal, std_form)

(variables = MathOptInterface.BasisStatusCode[MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC], bounds = MathOptInterface.BasisStatusCode[MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC, MathOptInterface.BASIC], constraints = MathOptInterface.BasisStatusCode[MathOptInterface.BASIC, MathOptInterface.NONBASIC_AT_UPPER, MathOptInterface.BASIC, MathOptInterface.NONBASIC_AT_UPPER, MathOptInterface.NONBASIC_AT_LOWER, MathOptInterface.NONBASIC_AT_LOWER, MathOptInterface.NONBASIC_AT_LOWER], basic_cols = Bool[1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0])

In [39]:
basis.basic_cols

12-element BitVector:
 1
 1
 1
 1
 1
 1
 0
 1
 0
 0
 0
 0

In [67]:
basis.constraints

7-element Vector{MathOptInterface.BasisStatusCode}:
 BASIC::BasisStatusCode = 0
 NONBASIC_AT_UPPER::BasisStatusCode = 3
 BASIC::BasisStatusCode = 0
 NONBASIC_AT_UPPER::BasisStatusCode = 3
 NONBASIC_AT_LOWER::BasisStatusCode = 2
 NONBASIC_AT_LOWER::BasisStatusCode = 2
 NONBASIC_AT_LOWER::BasisStatusCode = 2

In [50]:
dict_m

Dict{Any, Any} with 8 entries:
  "U"     => [0.0 10.0 … 38.0 22.0; 38.0 7.0 … 0.0 0.0; 49.0 59.0 … 17.0 11.0]
  "e"     => [51.0; 42.0; 31.0;;]
  "pii"   => [1.2; 3.6; 2.7; 2.1;;]
  "V"     => [75.0 32.0 0.0; 34.0 50.0 18.0; … ; 0.0 12.0 47.0; 5.0 0.0 38.0]
  "F_alt" => [0.0; 0.25; 0.1; 1.1;;]
  "V_alt" => [0.0 0.0 32.0]
  "U_alt" => [13.0; 24.0; 0.0;;]
  "F"     => [10.0 1.66667 … 0.0 0.833333; 0.0 4.72222 … 0.277778 1.38889; 1.11…

https://ergo-code.github.io/HiGHS/dev/interfaces/julia/

https://ergo-code.github.io/HiGHS/dev/guide/basic/

https://docs.juliahub.com/HiGHS/WFic5/1.7.5/autodocs/

https://jump.dev/JuMP.jl/stable/background/algebraic_modeling_languages/

https://github.com/jump-dev/GLPK.jl/blob/master/test/MOI_wrapper.jl